In [49]:
## libraries

import requests
import datetime
import json
from collections import namedtuple

In [50]:
## Functions

def readAPIkey(file_path):
    try:
        with open(file_path, 'r') as file:
            api_key = file.read().strip()
        return api_key
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except IOError:
        print(f"Error: Unable to read file '{file_path}'.")

In [109]:
## constants

X_RapidAPI_Host = "cricket-live-data.p.rapidapi.com"

api_key_path = 'config/apiKey.txt'
X_RapidAPI_Key = readAPIkey(api_key_path)

today = datetime.date.today()
today_string = today.strftime("%Y-%m-%d")

ipl_series_id = 1430
 
team_name = "Delhi Capitals" ## delhi capitals
player_id = 84255 ## msdhoni

In [52]:
## API Enpoint

url = "https://cricket-live-data.p.rapidapi.com/fixtures-by-date/" + today_string
headers = {
	"X-RapidAPI-Key": X_RapidAPI_Key,
	"X-RapidAPI-Host": X_RapidAPI_Host
}

In [53]:
## get today's fixtures

response = requests.get(url, headers=headers)
json_data = response.json()
json_data_path = "fixtures.json" # change the path accordingly
with open(json_data_path, "w") as file:
    json.dump(json_data, file, indent=4)

In [54]:
## find out if there is a ${team_name} match today

match_today = 0 ## changes to 1 if there is a match today
match_time_330 = 0
match_time_730 = 0
match_today_list = []

with open(json_data_path, "r") as file:
    todays_matches = json.load(file)

In [55]:
matchInfo = namedtuple('matchInfo', ['id', 'team1', 'team2', 'date', 'time'])

In [56]:
for item in todays_matches['results']:
    if item['series_id'] == ipl_series_id:
        if item['home']['name'] == team_name or item['away']['name'] == team_name:
            match_today = 1
            match_date = today_string
            match_time = todays_matches['results'][5]['date'][11:19]
            match_info = matchInfo(item['id'], item['home']['name'], item['away']['name'], match_date, match_time)
            if match_time == '14:00:00':
                match_time_730 = 1
            else:
                match_time_330 = 1

print("match_today: ", match_today)
print("match_time_730: ", match_time_730)
print("match_time_330: ", match_time_330)

match_today:  1
match_time_730:  1
match_time_330:  0


In [57]:
match_info

matchInfo(id=2719089, team1='Punjab Kings', team2='Delhi Capitals', date='2023-05-17', time='14:00:00')

In [111]:
match_info.id

2719089

now get the live score for the match

In [106]:
## if match today = 1
match_id = "2719009"
url = "https://cricket-live-data.p.rapidapi.com/match/" + match_id
headers = {
	"X-RapidAPI-Key": X_RapidAPI_Key,
	"X-RapidAPI-Host": X_RapidAPI_Host
}
response = requests.get(url, headers=headers)
json_data = response.json()

In [107]:
list_batsmen = []
try:
    scorecard = json_data['results']['live_details']['scorecard']
except:
    scorecard = None

if scorecard != None:
    for item1 in scorecard:
        for item2 in item1['batting']:
            list_batsmen.append(item2['player_id'])

list_batsmen

[3181152,
 1137432,
 833760,
 2143365,
 99435,
 26763,
 704037,
 84255,
 761418,
 134496,
 2559807,
 975090,
 3479145,
 90147,
 3251565,
 796704,
 2353149]

In [110]:
if player_id in list_batsmen:
    print("he is out to bat")

he is out to bat
